In [1]:
import pandas as pd
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
# More Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

import os

In [72]:
data_set = pd.read_csv("FootballDataEurope.csv")

In [73]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              25979 non-null  int64  
 1   country_name    25979 non-null  object 
 2   league_name     25979 non-null  object 
 3   season          25979 non-null  object 
 4   stage           25979 non-null  int64  
 5   date            25979 non-null  object 
 6   home_team       25979 non-null  object 
 7   away_team       25979 non-null  object 
 8   home_team_goal  25979 non-null  int64  
 9   away_team_goal  25979 non-null  int64  
 10  B365H           22592 non-null  float64
 11  B365D           22592 non-null  float64
 12  B365A           22592 non-null  float64
 13  BSH             14161 non-null  float64
 14  BSD             14161 non-null  float64
 15  BSA             14161 non-null  float64
 16  diff_goals      25979 non-null  int64  
 17  target          25979 non-null 

In [74]:
data_set.describe()

,id,stage,home_team_goal,away_team_goal,B365H,B365D,B365A,BSH,BSD,BSA,diff_goals,target
count,25979.000000,25979.000000,25979.000000,25979.000000,22592.000000,22592.000000,22592.000000,14161.000000,14161.000000,14161.000000,25979.000000,25979.000000
mean,12990.000000,18.242773,1.544594,1.160938,2.628818,3.839684,4.662222,2.497894,3.660742,4.405663,0.383656,0.746103
std,7499.635658,10.407354,1.297158,1.142110,1.794463,1.118269,3.730104,1.507793,0.868272,3.189814,1.782403,0.435248
min,1.000000,1.000000,0.000000,0.000000,1.040000,1.400000,1.080000,1.040000,1.330000,1.120000,-9.000000,0.000000
25%,6495.500000,9.000000,1.000000,0.000000,1.670000,3.300000,2.500000,1.670000,3.250000,2.500000,-1.000000,0.000000
50%,12990.000000,18.000000,1.000000,1.000000,2.100000,3.500000,3.500000,2.100000,3.400000,3.400000,0.000000,1.000000
75%,19484.500000,27.000000,2.000000,2.000000,2.800000,4.000000,5.250000,2.620000,3.750000,5.000000,1.000000,1.000000
max,25979.000000,38.000000,10.000000,9.000000,26.000000,17.000000,51.000000,17.000000,13.000000,34.000000,10.000000,1.000000


In [61]:
data_set["country_name"].value_counts()

Spain          3040
England        3040
France         3040
Italy          3017
Netherlands    2448
Germany        2448
Portugal       2052
Poland         1920
Scotland       1824
Belgium        1728
Switzerland    1422
Name: country_name, dtype: int64

In [71]:
data_set.head(1)
data_set["result"] = data_set.where(data_set["home_team_goal"] > data_set["away_team_goal"],"W","L")
#data_set["result"] = data_set.where(data_set["home_team_goal"] == data_set["away_team_goal"],"D",inplace=True)
#data_set["result"] = data_set.where(data_set["home_team_goal"] < data_set["away_team_goal"],"L",inplace=True)

#data_set["result"] = np.where(data_set["home_team_goal"] )
#data_set["result"] = np.where(data_set["home_team_goal"] == data_set["away_team_goal"],"D","W")

ValueError: For argument "inplace" expected type bool, received type str.

In [56]:
#Creating data frames of countries which we will be focusing on
esp = data_set[data_set["country_name"]== "Spain"]
eng = data_set[data_set["country_name"]== "England"]
frc = data_set[data_set["country_name"]== "France"]
itl = data_set[data_set["country_name"]== "Italy"]


In [57]:
eng

,id,country_name,league_name,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,B365H,B365D,B365A,BSH,BSD,BSA,diff_goals,target,result
58,1730,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Arsenal,West Bromwich Albion,1,0,1.20,6.50,15.00,1.22,5.00,13.00,1,1,1730
59,1731,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Sunderland,Liverpool,0,1,5.50,3.60,1.67,4.50,3.40,1.73,-1,1,L
60,1732,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,West Ham United,Wigan Athletic,2,1,1.91,3.40,4.20,1.91,3.25,3.80,1,1,1732
61,1734,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Everton,Blackburn Rovers,2,3,2.00,3.30,4.00,2.00,3.25,3.50,-1,1,L
62,1735,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Middlesbrough,Tottenham Hotspur,2,1,3.20,3.40,2.25,2.80,3.25,2.30,1,1,1735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25942,4705,England,England Premier League,2015/2016,38,2016-05-15 00:00:00,Stoke City,West Ham United,2,1,3.50,3.60,2.15,NaN,NaN,NaN,1,1,4705
25943,4706,England,England Premier League,2015/2016,38,2016-05-15 00:00:00,Swansea City,Manchester City,1,1,6.00,4.75,1.53,NaN,NaN,NaN,0,0,L
25944,4707,England,England Premier League,2015/2016,38,2016-05-15 00:00:00,Watford,Sunderland,2,2,2.05,3.75,3.70,NaN,NaN,NaN,0,0,L
25945,4708,England,England Premier League,2015/2016,38,2016-05-15 00:00:00,West Bromwich Albion,Liverpool,1,1,2.40,3.60,3.00,NaN,NaN,NaN,0,0,L
